In [1]:
# imports
from scipy.spatial import Voronoi, voronoi_plot_2d
from matplotlib.collections import PatchCollection
from matplotlib import animation, pyplot as plt, gridspec
from matplotlib.patches import Polygon
import matplotlib.patches as mpatches
from shapely.geometry import Polygon as ShapelyPolygon, box
import numpy as np
import pandas as pd
import psycopg2
import dotenv
import os
from matplotlib import animation
from matplotlib import pyplot as plt
from IPython.display import HTML
import dotenv
import os
import pandas as pd
from mplsoccer import Pitch

ModuleNotFoundError: No module named 'shapely'

In [ ]:


dotenv.load_dotenv()

PG_PASSWORD = os.getenv("PG_PASSWORD")
PG_USER = os.getenv("PG_USER")
PG_HOST = os.getenv("PG_HOST")
PG_PORT = os.getenv("PG_PORT")
PG_DATABASE = os.getenv("PG_DB")

# Database connection parameters
import psycopg2
import os
from scipy.interpolate import interp1d

conn = psycopg2.connect(
    host=PG_HOST,
    database=PG_DATABASE,
    user=PG_USER,
    password=PG_PASSWORD,
    port=PG_PORT,
    sslmode="require",
)


In [2]:
def get_possession_change_data(conn, game_id):

    possession_query = """
    WITH action_changes AS (
        SELECT
            a.*,
            LAG(a.team_id) OVER (ORDER BY a.period_id, a.seconds, a.id) AS prev_team_id,
            LEAD(a.team_id) OVER (ORDER BY a.period_id, a.seconds, a.id) AS next_team_id
        FROM
            spadl_actions a
        WHERE
            a.game_id = %s
    ),
    possession_markers AS (
        SELECT
            *,
            CASE 
                WHEN prev_team_id IS NULL OR team_id != prev_team_id 
                THEN 1 
                ELSE 0 
            END AS is_new_possession
        FROM
            action_changes
    ),
    possession_sequences AS (
        SELECT
            *,
            SUM(is_new_possession) OVER (ORDER BY period_id, seconds, id) AS possession_group
        FROM
            possession_markers
    ),
    possession_stats AS (
        SELECT
            possession_group,
            team_id,
            COUNT(*) AS action_count,
            MAX(id) AS last_action_id
        FROM
            possession_sequences
        GROUP BY
            possession_group, team_id
    )
    SELECT 
        s.id,
        s.period_id,
        s.seconds,
        s.action_type,
        s.team_id AS losing_team_id,
        s.next_team_id AS gaining_team_id,
        t1.team_name AS losing_team,
        t2.team_name AS gaining_team,
        s.start_x,
        s.start_y,
        s.end_x,
        s.end_y,
        ps.action_count
    FROM
        possession_sequences s
    JOIN
        possession_stats ps ON s.possession_group = ps.possession_group 
                             AND s.team_id = ps.team_id
                             AND s.id = ps.last_action_id
    JOIN teams t1 ON s.team_id = t1.team_id
    JOIN teams t2 ON s.next_team_id = t2.team_id
    WHERE
        ps.action_count >= 3
        AND s.team_id != s.next_team_id
        AND s.next_team_id IS NOT NULL
    ORDER BY
        s.period_id, s.seconds
    """

    possession_changes = pd.read_sql_query(possession_query, conn, params=(game_id,))
    possession_changes['match_id'] = game_id
    return possession_changes

df_possession = get_possession_change_data(conn, '5pcyhm34h5c948yji4oryevpw')
df_possession

C:\Users\axlnu\AppData\Local\Temp\ipykernel_19588\2135716486.py:73: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  possession_changes = pd.read_sql_query(possession_query, conn, params=(game_id,))


,id,period_id,seconds,action_type,losing_team_id,gaining_team_id,losing_team,gaining_team,start_x,start_y,end_x,end_y,action_count,match_id
0,2071,1,4.0,0,cyrrlv6l1onld5x247w1q1jlr,bw9wm8pqfzcchumhiwdt2w15c,Beerschot,OH Leuven,38.115,24.752,73.185,11.356,4,5pcyhm34h5c948yji4oryevpw
1,2079,1,21.0,0,cyrrlv6l1onld5x247w1q1jlr,bw9wm8pqfzcchumhiwdt2w15c,Beerschot,OH Leuven,66.150,20.604,73.290,16.524,7,5pcyhm34h5c948yji4oryevpw
2,2098,1,53.0,19,cyrrlv6l1onld5x247w1q1jlr,bw9wm8pqfzcchumhiwdt2w15c,Beerschot,OH Leuven,101.430,60.316,101.430,60.316,16,5pcyhm34h5c948yji4oryevpw
3,2102,1,85.0,0,bw9wm8pqfzcchumhiwdt2w15c,cyrrlv6l1onld5x247w1q1jlr,OH Leuven,Beerschot,59.220,11.764,53.655,19.176,4,5pcyhm34h5c948yji4oryevpw
4,2138,1,142.0,7,cyrrlv6l1onld5x247w1q1jlr,bw9wm8pqfzcchumhiwdt2w15c,Beerschot,OH Leuven,37.275,1.632,37.275,1.632,36,5pcyhm34h5c948yji4oryevpw
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,3725,2,2754.0,0,bw9wm8pqfzcchumhiwdt2w15c,cyrrlv6l1onld5x247w1q1jlr,OH Leuven,Beerschot,100.590,41.888,92.085,68.000,3,5pcyhm34h5c948yji4oryevpw
173,3744,2,2789.0,7,bw9wm8pqfzcchumhiwdt2w15c,cyrrlv6l1onld5x247w1q1jlr,OH Leuven,Beerschot,59.430,4.488,59.430,4.488,18,5pcyhm34h5c948yji4oryevpw
174,3758,2,2822.0,0,bw9wm8pqfzcchumhiwdt2w15c,cyrrlv6l1onld5x247w1q1jlr,OH Leuven,Beerschot,41.370,0.204,33.600,2.924,13,5pcyhm34h5c948yji4oryevpw
175,3766,2,2842.0,7,bw9wm8pqfzcchumhiwdt2w15c,cyrrlv6l1onld5x247w1q1jlr,OH Leuven,Beerschot,35.595,65.280,35.595,65.280,3,5pcyhm34h5c948yji4oryevpw


In [3]:
def get_tracking_data(match_id):
    """Get player and ball tracking data for a time window."""

    # Query for player positions
    player_query = """
    SELECT 
        pt.frame_id,
        pt.timestamp,
        pt.period_id,
        pt.player_id,
        p.player_name,
        p.jersey_number,
        p.team_id,
        pt.x,
        pt.y
    FROM 
        player_tracking pt
    JOIN 
        players p ON pt.player_id = p.player_id
    WHERE 
        pt.game_id = %s
    ORDER BY 
        pt.frame_id, p.team_id
    """
    
    # Query for ball positions
    ball_query = """
    SELECT 
        pt.frame_id,
        pt.timestamp,
        pt.period_id,
        'ball' as player_id,
        NULL as jersey_number,
        NULL as team_id,
        pt.x,
        pt.y
    FROM 
        player_tracking pt
    WHERE 
        pt.game_id = %s
        AND pt.player_id = 'ball'
    ORDER BY 
        pt.frame_id
    """
    
    # Execute queries with proper parameter passing
    players_df = pd.read_sql_query(
        player_query, 
        conn, 
        params=[match_id]
    )
    
    ball_df = pd.read_sql_query(
        ball_query, 
        conn, 
        params=[match_id]
    )
    
    if players_df.empty or ball_df.empty:
        print(f"No data found for match_id: {match_id}")
        return pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
    
    # Get team information
    team_ids = players_df['team_id'].unique()
    if len(team_ids) >= 2:
        home_team_id = players_df['team_id'].value_counts().index[0]
        away_team_id = players_df['team_id'].value_counts().index[1]
        
        # Split players by team
        home_df = players_df[players_df['team_id'] == home_team_id].copy()
        away_df = players_df[players_df['team_id'] == away_team_id].copy()
        
        # Add Frame column for consistency
        home_df['Frame'] = home_df['frame_id']
        away_df['Frame'] = away_df['frame_id']
        ball_df['Frame'] = ball_df['frame_id']

        home_df['timestamp'] = pd.to_timedelta(home_df['timestamp']).dt.total_seconds().astype(float)
        away_df['timestamp'] = pd.to_timedelta(away_df['timestamp']).dt.total_seconds().astype(float)
        ball_df['timestamp'] = pd.to_timedelta(ball_df['timestamp']).dt.total_seconds().astype(float)
        return home_df, away_df, ball_df
    else:
        print(f"Not enough teams found for match_id: {match_id}")
        return pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

In [4]:
def display_Animation(players_df, ball_df):
    pitch = Pitch(pitch_type='metricasports', goal_type='line', pitch_width=68, pitch_length=105)
    fig, ax = pitch.draw(figsize=(16, 10.4))


    marker_kwargs = {'marker': 'o', 'markeredgecolor': 'black', 'linestyle': 'None'}
    ball, = ax.plot([], [], ms=6, markerfacecolor='w', zorder=3, **marker_kwargs)
    away, = ax.plot([], [], ms=10, markerfacecolor='#b94b75', **marker_kwargs) 
    home, = ax.plot([], [], ms=10, markerfacecolor='#7f63b8', **marker_kwargs)  

    def animate(i):
        """ Function to animate the data. Each frame it sets the data for the players and the ball."""
        # set the ball data with the x and y positions for the ith frame
        ball.set_data([ball_df.iloc[i]['x']/100],
                    [ball_df.iloc[i]['y']/100])
    
        frame = ball_df.iloc[i,0]

        frame_players = players_df[players_df['Frame'] == frame].dropna(subset=['x', 'y'])

        home_players = frame_players[frame_players['team']=='Home']
        away_players = frame_players[frame_players['team']=='Away']
        
    

        # set the player data using the frame id
        away.set_data(home_players['x']/100,
                    home_players['y']/100)
        home.set_data(away_players['x']/100,
                    away_players['y']/100)
       
        return ball, away, home


    # call the animator, animate so 25 frames per second
    anim = animation.FuncAnimation(fig, animate, frames=len(ball_df), interval=20, blit=True)
    HTML(anim.to_jshtml())

In [5]:
### Note for me: THE TIME IN SECONDS IS FOR EACH PERIOD, NOT TOTAL TIME
home_movement = pd.DataFrame()
away_movement = pd.DataFrame()
ball_movement = pd.DataFrame()
for _, row in df_possession.iterrows():
    
    match_id = row['match_id']
    start_time = row['seconds']
    end_time = start_time + 5  
    home_df, away_df, ball_df = get_tracking_data(match_id)
    
    if not home_df.empty and not away_df.empty and not ball_df.empty:
        home_df = home_df[(home_df['timestamp'] >= start_time) & (home_df['timestamp'] <= end_time)]
        away_df = away_df[(away_df['timestamp'] >= start_time) & (away_df['timestamp'] <= end_time)]
        ball_df = ball_df[(ball_df['timestamp'] >= start_time) & (ball_df['timestamp'] <= end_time)]
        home_df['team'] = 'Home'
        away_df['team'] = 'Away'
        players_df  = pd.concat([home_df, away_df])
        home_movement = home_df[['Frame', 'x', 'y']].copy()
        away_movement = away_df[['Frame', 'x', 'y']].copy()
        ball_movement = ball_df[['Frame', 'x', 'y']].copy()
        home_movement['Possesion_Transition'] = row.index
        away_movement['Possesion_Transition'] = row.index
        ball_movement['Possesion_Transition'] = row.index
        home_movement['team'] = 'Home'
        away_movement['team'] = 'Away'
        ball_movement['team'] = 'Ball' 
    else:
        print(f"Data retrieval failed for match_id: {match_id}, start_time: {start_time}")

players_movement = pd.concat([home_movement, away_movement])
display_Animation(players_movement, ball_movement)


C:\Users\axlnu\AppData\Local\Temp\ipykernel_19588\2766514525.py:47: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  players_df = pd.read_sql_query(
C:\Users\axlnu\AppData\Local\Temp\ipykernel_19588\2766514525.py:53: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ball_df = pd.read_sql_query(


ValueError: Length of values (14) does not match length of index (121)